In [1]:
# Imports Training Data
import sys
sys.path.append('../lambda/data')

from tweet_ner_data_label import train_data

In [2]:
# Imports & Dependencies
from __future__ import unicode_literals, print_function

import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


In [3]:
print(spacy.__version__)

2.3.5


In [4]:
# Update original data to include Normal NER Tags to prevent forgetting problem
nlp = spacy.load('en_core_web_md')

In [5]:
# Checks entities if multiple truck locations
def count_locations(ents_arr):
    count = 0
    
    for i in range(0,len(ents_arr)):
        cur_ent_type = ents_arr[i][2]
        if cur_ent_type == 'TRUCK_LOCATION':
            count += 1
    
    return count

#MODIFY THIS
# NO OVERLAPS WITH TRUCK DATA
    
# Adds old NER tags to train data
def append_old_ner(train_data):
    for i in range(0,len(train_data)):
        doc = nlp(train_data[i][0])
        ents_arr = train_data[i][1]['entities']
        locations_count = count_locations(ents_arr)

        for ent in doc.ents:
            start = int(ent.start_char)
            end = int(ent.end_char)

            # Cases
            # < Start---End > Bound entire word and more
            case1 = lambda truck_start, truck_end: start <= truck_start and end >= truck_end
            # < Start>---End< Bound entire left up to end
            case2 = lambda truck_start, truck_end: start <= truck_start and end <= truck_end and end >= truck_start
            # >Start---<End > Bound entire right up to start
            case3 = lambda truck_start, truck_end: end >= truck_end and start >= truck_start and start <= truck_end
            
            if locations_count == 0:
                ents_arr.append((start, end, ent.label_))
            else:
                continue_outer = False
                for j in range(0,locations_count):
                    truck_start = int(ents_arr[j][0])
                    truck_end = int(ents_arr[j][1])
                    
                    if case1(truck_start, truck_end) or case2(truck_start, truck_end) or case3(truck_start, truck_end):
                        continue_outer = True
                        break
                
                if continue_outer:
                    continue
                else:
                    ents_arr.append((start, end, ent.label_))
                    
    return train_data

train_data = append_old_ner(train_data)

In [8]:
# Validate old NER Tags
print(train_data[105])

('#tour de fat today. Come down to Yards park and try your favorite Cuban speciality paired with New Belgium beer.', {'entities': [(33, 43, 'TRUCK_LOCATION'), (0, 12, 'PERSON'), (13, 18, 'DATE'), (66, 71, 'NORP'), (95, 106, 'GPE')]})


In [13]:
# Load model if exists
def load_model(model=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    return nlp

# List of Names Entities
# To be referenced later on
move_names = []

# Train update Spacy's NER
def train_NER(train_data, iterations, model=None):
    TRAIN_DATA = train_data
    
    # Set language to only Text in tweets
    nlp = load_model(model)
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')
        
    # add new labels eg: using TRUCK_LOCATION
    for _, annotations in TRAIN_DATA:
        print(_)
        print(annotations)
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)
            
    return nlp

In [14]:
# Train Data
nlp = train_NER(train_data, 5)

Created blank 'en' model
We have a Sunday full of markets and open houses tomorrow. Check out this gem in Falls Church tomorrow (11/17) 2-4p… https://t.co/RSIpjA30NH
{'entities': [(81, 93, 'TRUCK_LOCATION'), (10, 16, 'DATE'), (49, 57, 'DATE'), (94, 102, 'DATE'), (104, 109, 'CARDINAL')]}
Sorry people. Because the rain isn't letting up we won't be coming out today, See you next week. HAVE A MERRY CHRISTMAS!
{'entities': [(71, 76, 'DATE'), (86, 95, 'DATE'), (104, 119, 'DATE')]}
Hey, LA! Catch the Astro Truck in today from 11-2 in Playa Del Rey! https://t.co/fT83xI7E4D
{'entities': [(34, 39, 'DATE'), (45, 47, 'CARDINAL'), (53, 66, 'GPE')]}
Waterfront Metro Station is where you can find us today! Come try our wide variety of foods! Hope to see you all here!!!
{'entities': [(0, 24, 'TRUCK_LOCATION'), (50, 55, 'DATE')]}
Find our schedule and menu on @Goodfynd!📅 Wed, Dec 11🕚 11a - 1:30p📍 Metro Center Station🍽… https://t.co/JCAIiatxZ4
{'entities': [(71, 91, 'TRUCK_LOCATION'), (47, 53, 'DATE'), 

RT @KaftaMania: Good luck to my new family @DMVFTA at the hearing today. Thank you guys for all that you have done for us and all food truc…
{'entities': [(0, 2, 'ORG'), (43, 50, 'NORP'), (66, 71, 'DATE')]}
RT @WestfieldMON: Deal of the Day @JMCCurbside Curry Chicken meal &amp; a drink ($8.00) or Coco Bread with your choice patty (beef/chicken) &amp;…
{'entities': [(0, 2, 'ORG'), (18, 33, 'PERSON'), (47, 60, 'ORG'), (82, 86, 'MONEY')]}
Thanks  Kido https://t.co/AMeUHss4sK
{'entities': [(0, 36, 'PERSON')]}
TODAY:- METRO CENTER - BALLSTON
{'entities': [(8, 20, 'TRUCK_LOCATION'), (23, 31, 'TRUCK_LOCATION')]}
The Mesob at Friendship Heights #Vegan #chickpeastew #yellowsplitpeastew #spicylentilstew and more.  #spicybeefstew… https://t.co/SJzs3AqTbn
{'entities': [(13, 31, 'TRUCK_LOCATION')]}
Hello Italian and wine lovers today we will be wineryatbullrun come on out and try to delicious… https://t.co/XIdLFPyS34
{'entities': [(47, 62, 'TRUCK_LOCATION'), (6, 13, 'NORP'), (30, 35, 'DATE')]}
It i

{'entities': [(29, 43, 'TRUCK_LOCATION'), (5, 12, 'TIME'), (14, 19, 'DATE')]}
Come to DC w/#CheffJeffCitco gas station carry-out!!1905a 9th st. NE Washington DC&amp; dont forget? ? ? Yougotthatf… https://t.co/pdL836t9UI
{'entities': [(52, 79, 'TRUCK_LOCATION'), (8, 10, 'GPE'), (58, 61, 'ORDINAL'), (62, 68, 'GPE'), (105, 116, 'PERSON')]}
Do you remember when you joined Twitter? I do! #MyTwitterAnniversary https://t.co/hav39GuC50
{'entities': [(32, 39, 'ORG'), (47, 92, 'PERSON')]}
Happy Monday! A great way to start off the week is with a delicious lunch! You can find us at L'Enfant Plaza Station.
{'entities': [(94, 116, 'TRUCK_LOCATION'), (6, 12, 'DATE'), (39, 47, 'DATE')]}
Bringing the smoothies to Ballston today!!!
{'entities': [(26, 34, 'TRUCK_LOCATION'), (35, 40, 'DATE')]}
Today 13 &amp; k st for lunch
{'entities': [(6, 19, 'TRUCK_LOCATION')]}
Near 963-995 13th Street Northwest in Washington
{'entities': [(5, 34, 'TRUCK_LOCATION'), (13, 17, 'ORDINAL'), (38, 48, 'GPE')]}
Farragut we g

{'entities': []}
Popup this week! #sloppyjoes #neatmeat #dc #sloppyjoy #dcfoodies #dcfoodtrucks #foodtrucks #yum #yumdc #dcigers… https://t.co/Ufc2R2l5rQ
{'entities': [(0, 5, 'ORG'), (6, 15, 'DATE'), (79, 90, 'MONEY'), (91, 95, 'PERSON'), (96, 102, 'PERSON'), (113, 136, 'ORG')]}
Reuben! https://t.co/MXJCZd2d16
{'entities': [(0, 6, 'PERSON'), (8, 31, 'PERSON')]}
Today we are serving L'Enfant  plaza come enjoy you're Lunch with us .
{'entities': [(21, 36, 'TRUCK_LOCATION'), (0, 5, 'DATE')]}
RT @FabbioliWines: Our 12' Tannat won Gold at the Virginia Governor's Cup!! #vawine http://t.co/9wLBnLTt73
{'entities': [(0, 2, 'ORG'), (23, 25, 'CARDINAL'), (27, 73, 'WORK_OF_ART'), (76, 83, 'PERSON')]}
Dear loyal GAHD Fans, As many of you know, we had a gas leak at the restaurant and had to temporarily close until… https://t.co/wDy6z0OLaS
{'entities': [(11, 15, 'PERSON')]}
RT @FWB_QuarterDeck: What a beautiful day here at @FairWindsBrew! @quickwraps is here and the beer is flowing https://t.co/l7vS7

/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  **kwargs
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  **kwargs
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "RT @Med_Delight: Join us @MaseratiArl FOOD TRUCK R..." with entities "[(26, 37, 'TRUCK_LOCATION'), (0, 2, 'ORG'), (55, 5...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the ali

ValueError: [E103] Trying to set conflicting doc.ents: '(29, 61, 'TRUCK_LOCATION')' and '(34, 40, 'ORG')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.

In [12]:
# Review NER Results
def show_ents(doc):
    doc = nlp(doc[0])
    if doc.ents:
        for ent in doc.ents:
            print(ent.text + ' - ' + ent.label_)
    else:
        print('No entities found')

In [13]:
# Test across subset of tweets
# for i in range(0,10):
#     show_ents(train_data[i])
#     print("")
    
print(train_data[74])
print('')
show_ents(train_data[74])

('Good morning NoMA today open Now', {'entities': [(13, 18, 'TRUCK_LOCATION')]})

morning - TIME
NoMA - TRUCK_LOCATION
today - DATE


In [14]:
# Manually test the trained model
def test_model(nlp, test_text):
    doc = nlp(test_text)
    print("Entities in '%s' \n" % test_text )
    for ent in doc.ents:
        print(ent.label_, ent.text)
        
test_model(nlp, "Today we are at 2nd Ave Parrish")

Entities in 'Today we are at 2nd Ave Parrish' 

DATE Today
TRUCK_LOCATION 2nd Ave Parrish


In [15]:
# Save model to output directory
def save_model(nlp, output_dir, name):
    if output_dir is not None:
        output_dir = Path(output_dir)
        
        if not output_dir.exists():
            output_dir.mkdir()
            
        nlp.meta[name] = name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

def test_saved_model(model_dir, test_text):
    # test the saved model
    print("Loading from", model_dir)
    nlp = spacy.load(model_dir)

    # Check the classes have loaded back consistently
    # assert nlp.get_pipe("ner").move_names == move_names
    if nlp.get_pipe('ner').move_names[0] != 'B-TRUCK_LOCATION':
        print('NER Pipe doesn\'t have Truck Location')
        return
    
    test_model(nlp, test_text)

In [17]:
save_model(nlp,'../lambda/model/tweet_ner_model','ner_1')

Saved model to ../lambda/model/tweet_ner_model


In [18]:
test_saved_model('../model/tweet_ner_model', "Today we are at 2nd Ave Parrish")

Loading from ../model/tweet_ner_model


OSError: [E050] Can't find model '../model/tweet_ner_model'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [1]:
# Imports & Dependencies
from __future__ import unicode_literals, print_function

from numpy import load

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight

import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

# Data
from tweet_data_label import train_data

# Spacy Load
nlp = spacy.load('en')

/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/farris

693
Tweet 1
'This is my f**king country': Racist white woman arrested for attacking passengers on New York bus https://t.co/Rfm0jauh72
_____

Tweet 2
RT @mussie671: Freestyle #3, Fiasco type shiiii https://t.co/0GTJscWORW
_____

Tweet 3
What you think Cole? @JColeNC @JColeDirect https://t.co/oE4Ipvq6hL
_____

Tweet 4
⚡️ “Redskins dominate the Raiders on Sunday Night Football ”https://t.co/4VuHWOs1gX
DATE: Sunday Night 
_____

Tweet 5
RT @NWSSanJuan: 215PM FLASH FLOOD EMERGENCY for A Dam Failure in Isabela Municipality y Quebradillas Municipality in Puerto Rico... #prwx h…
NEIGHBORHOOD: Isabela Municipality NEIGHBORHOOD: Quebradillas Municipality 
TIME: 215PM 
_____

Tweet 6
Posted @chinatown on the corner of 7th &amp; D st. NW, PERFECT WEATHER TO SLIDE UP!!
ADDRESS: 7th &amp; D st. NW 

POSITIVE: Posted @chinatown on the corner of 
_____

Tweet 7
Good Morning DMV, we are @ChinaTown on the corner of 7th &amp; G st NW. Hand crafted burgers, endless toppings, Hand cut fries,  PULL UP !!!



Tweet 310
RT @truckeroo: Tomorrow is the last Truckeroo of the season! Come out to celebrate the start of an amazing weekend. See you there!Trucker…

NEGATIVE
_____

Tweet 311
We have missed ALL our fans in #Tysons speared the news we are there for lunch tomorrow #vegan #Egyptian #yummy… https://t.co/uviH5VhAL1

NEGATIVE
_____

Tweet 312
Good morning! Yellow Tin heads towards to US ICE today. See you all during lunch hour.

POSITIVE
_____

Tweet 313
Good morning Navy Yardian! Yellow Tin landed in front of Five Guys. See you all during lunch hour.

POSITIVE
_____

Tweet 314
Good morning! Yellow Tin heads towards to Navy Yard today. See you all during lunch hour.

POSITIVE
_____

Tweet 315
Good morning! Yellow Tin heads towards to Metro Center today. See you all during lunch hour.

POSITIVE
_____

Tweet 316
Good morning! Yellow Tin heads towards to US ICE today. See you all during lunch hour.

POSITIVE
_____

Tweet 317
Good morning! Yellow Tin landed at Farragut Square to serve lunch to

_____

Tweet 595
Monday we will be at Union station with fresh Bibim bap, Bul goki with Jap chae.  See you~

NEGATIVE
_____

Tweet 596
Tuesday we will be at Union station with fresh Korean food, Bibim bap, Bul goki with Jap chae. See you tomorrow at lunch time.

NEGATIVE
_____

Tweet 597
Thursday we will be at 500 12th St. SW in front of ICE with fresh Korean food for your lunch. Stay warm and see you tomorrow.

NEGATIVE
_____

Tweet 598
Good morning Clarendon! Unfortunately we can't make it today, parking is full.

NEGATIVE
_____

Tweet 599
Franklin square Thursday. Can't wait to serve fresh Korean food to you then. See you~

NEGATIVE
_____

Tweet 600
Union station Satuday. Can't find parking today. Open window at 11 tomorrow. See you~

NEGATIVE
_____

Tweet 601
Friday we will be at L'Enfant plaza with fresh and delicious Korean food. You need some hot and spicy food for this cold week. See you~

NEGATIVE
_____

Tweet 602
Thursday we will be at Franklin square with fresh Bibim bap, Bu

In [22]:
class DataTokenizer:
    def __init__(self, train_data):
        self.tokenizer = Tokenizer()
        self.train_data = train_data

        # Sets up tokenizer
        try:
            self.__init_tokenizer()
        except ModuleNotFoundError:
            raise

    # Clean data
    def __tweet_clean(self, text):
        lower = []
        text = text.replace('&amp;', 'and')

        for token in nlp(text):
            lower.append(token.text.lower())

        return lower
            
    # Orginize Tweet Data for tokenizer
    def __organize_tweet_data(self):
        tweet_sequences = []

        # Organize tweets & train data into arrays
        for tweet_data in self.train_data:
            tweet_sequences.append(self.__tweet_clean(tweet_data[0]))

        return tweet_sequences
    
    def __init_tokenizer(self):
        data = self.__organize_tweet_data()
        self.tokenizer.fit_on_texts(data)
        
    # Tokenization Methods
    def tokenize_tweets(self, data):
        self.tokenizer.fit_on_texts(data)

        return self.tokenizer.texts_to_sequences(data)

class NERModel:
    def __init__(self, ner_model_path):
        try:
            self.nlp_ner = spacy.load(ner_model_path)
        except ModuleNotFoundError:
            raise
            
    def predict(self, text):
        # Check the classes have loaded back consistently
        # assert nlp.get_pipe("ner").move_names == move_names
        if self.nlp_ner.get_pipe('ner').move_names[0] != 'B-TRUCK_LOCATION':
            print('NER Pipe doesn\'t have Truck Location')
            return
        
        doc = self.nlp_ner(text)
        
        if doc.ents:
            return doc.ents
        else:
            return []
    
class AffirmationModel:
    def __init__(self, affirmation_model_path, scaler_data_path, train_data, max_len):
        self.max_len = max_len # Max Tweet Length
        
        try:
            self.scaler_data = self.__load_scaler_data(scaler_data_path, 'scaler_data.npy')
        except ModuleNotFoundError:
            raise
        
        # Sets up tokenizer
        try:
            # Setup Tokenizer
            self.DataTokenizer = DataTokenizer(train_data)
            self.tokenizer = self.DataTokenizer.tokenizer
        except ModuleNotFoundError:
            raise
            
        try:
            self.nlp_affirmation = keras.models.load_model(affirmation_model_path, compile=False)
        except ModuleNotFoundError:
            raise
            
    def predict(self,tweet):
        # Predict affirmation
        tokenized_tweets = self.DataTokenizer.tokenize_tweets([self.__tweet_lower(tweet)])
        predict_tweets = [self.__pad_array(data, self.max_len) for data in tokenized_tweets]
        scaled_test_data = self.__scale_test_data(predict_tweets)
        affirmation_prediction = self.nlp_affirmation.predict_classes(scaled_test_data)
        
        if affirmation_prediction[0] == 0:
            return True
        else:
            return False
    
    # Tweet lower
    def __tweet_lower(self, text):
        text = text.replace('&amp;', 'and')

        return text.lower()
        
    def __create_zeros_array(self, length):
        zeros_arr = []

        i = 0
        while i < length:
            zeros_arr.append(0)
            i += 1

        return zeros_arr

    def __pad_array(self,data, max_len):
        zeros_len = self.max_len - len(data)
        zeros_arr = self.__create_zeros_array(zeros_len)

        return [*data,*zeros_arr]
    
    def __load_scaler_data(self, directory, file):
        try:
            path = f'{directory}/{file}'
            data = load(path)
            
            return data
        except ModuleNotFoundError:
            raise
            
    def __scale_test_data(self, data):
        scaler_object = MinMaxScaler()
        scaler_object.fit(self.scaler_data)
        scale_data = scaler_object.transform(data)
        
        return scale_data

class AssertModel:
    def __init__(self, affirmationModel, nerModel):
        self.affirmationModel = affirmationModel
        self.nerModel = nerModel

    def predict(self, tweet):
        affirmation = self.affirmationModel.predict(tweet)
        named_entities = self.nerModel.predict(tweet)
 
        location_types = {
            'TRUCK_LOCATION': [],
            'FAC': [],
            'GPE': [],
            'LOC': [],
        }
        
        for ent in named_entities:
            if ent.label_ in location_types:
                location_types[ent.label_].append(ent.text)
        
        result = {
            'tweet': tweet,
            'affirmation': affirmation,
            'locations': location_types
        }
        
        return result

In [23]:
affirmationModel = AffirmationModel('./tweet_affirmation_model/test', './tweet_affirmation_model', train_data, 35)
nerModel = NERModel('./tweet_ner_model')
assertModel = AssertModel(affirmationModel, nerModel)

NameError: name 'load' is not defined

In [60]:
print(assertModel.predict("Today we are at 5th street"))

{'tweet': 'Today we are at 5th street', 'affirmation': True, 'locations': {'TRUCK_LOCATION': ['5th street'], 'FAC': [], 'GPE': [], 'LOC': []}}


In [ ]:
# Next steps, move & improve this into a properly formatted structure for AWS Lambda